In [105]:
import pandas as pd
import json
import datetime

In [107]:
file_path = "data/weather_trani/weather_data_consum.json"   # for the consumption data (wider time range)
# file_path = "data/weather_trani/weather_data.json"

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

In [108]:
data_weather = pd.DataFrame()
start_date = datetime.datetime(2018, 8, 1)


for el in data:
    data_temp = el['main']
    timestamp = start_date
    start_date += timedelta(days=1)
    temp = data_temp['temp']-273
    press = data_temp['pressure']
    humid = data_temp['humidity']
    new_row = {'Timestamp' : timestamp,'temp':temp,'press':press,'humid':humid}

    data_weather = data_weather.append(new_row, ignore_index=True)

C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_weather = data_weather.append(new_row, ignore_index=True)
C:\Users\abelo\AppData\Local\Temp\ipykernel_6712\1077695112.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat

In [109]:
data_weather

,Timestamp,temp,press,humid
0,2018-08-01,21.98,1006,55
1,2018-08-02,21.98,1006,55
2,2018-08-03,21.98,1006,55
3,2018-08-04,21.98,1006,55
4,2018-08-05,21.98,1006,55
...,...,...,...,...
1192,2021-11-05,21.98,1006,55
1193,2021-11-06,21.98,1006,55
1194,2021-11-07,21.98,1006,55
1195,2021-11-08,21.98,1006,55


Get the right Timestamp and compute the mean power value of consumption and production.

In [110]:
from datetime import datetime 

In [116]:
building_consumption = pd.read_csv("data/Office building.csv")
building_production = pd.read_csv("data/PV plant.csv")

print(f"building_consumption COLUMNS: {building_consumption.columns}")
print(f"building_production COLUMNS: {building_production.columns}")

data_series = []

for index, row in building_consumption.iterrows():
    timestamp_string = row['Timestamp']
    # Split the string into time and timezone parts
    time_part, timezone_part = timestamp_string.split("+")

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))

    #dt = datetime(row['Year'], row['Month'], row['Day'], hours, minutes, seconds)
    dt = datetime(row['Year'], row['Month'], row['Day'])

    data_series.append(dt)
building_consumption['Timestamp'] = data_series


avg_power_consum = building_consumption.groupby('Timestamp')['Power (W)'].mean()
avg_power_consum = (pd.DataFrame(avg_power_consum))

building_consumption COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)', 'Energy (Wh)'], dtype='object')
building_production COLUMNS: Index(['Year', 'Month', 'Day', 'Timestamp', 'Power (W)'], dtype='object')


In [117]:
data_series = []
for index, row in building_production.iterrows():
    timestamp_string = row['Timestamp']
    # Split the string into time and timezone parts
    time_part, timezone_part = timestamp_string.split("+")

    # Split the time part into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_part.split(":"))

    #dt = datetime(row['Year'], row['Month'], row['Day'], hours, minutes, seconds)
    dt = datetime(row['Year'], row['Month'], row['Day'])

    data_series.append(dt)
building_production['Timestamp'] = data_series

avg_power_product = building_production.groupby('Timestamp')['Power (W)'].mean()
avg_power_product = (pd.DataFrame(avg_power_product))

In [119]:
avg_power_consum

,Power (W)
Timestamp,
2018-06-12,14699.962069
2018-06-13,64037.939583
2018-06-14,92844.061806
2018-06-15,84867.941667
2018-06-16,67127.913194
...,...
2020-12-29,80175.475694
2020-12-30,76033.184722
2020-12-31,71350.627083


In [124]:
type(avg_power_consum.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [129]:

avg_power_product['Power (W)'] = avg_power_product['Power (W)'] + avg_power_consum[avg_power_consum.index.isin(avg_power_product.index)]['Power (W)']


In [130]:
df_merged_power_consum = avg_power_consum.merge(data_weather, on='Timestamp')
df_merged_power_product = avg_power_product.merge(data_weather, on='Timestamp')

In [131]:
df_merged_power_consum.to_csv('data/exported_merged_power_consum.csv')
df_merged_power_product.to_csv('data/exported_merged_power_product.csv')

Energy (Wh) consumption/production plotting